# Постановка проблемы
По моему мнению в данной задаче решается проблема мультиклассовой классификации с учителем.
Необходимо подобрать такой алгоритм, который на основании ранее полученных данных с известным ответом (обучение)
в дальнейшем будет понимать, к какому классу отнести вновь полученные данные.

# Решение
Для начала произведём необходимые импорты:

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import svm, tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Получаем входные данные:

In [ ]:
dataset = pd.read_csv('../dataset.csv', usecols=['Score', 'Action'])
dataset.head()

Построим график для визуального анализа данных:

In [ ]:
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(dataset, row='Action', hue='Action', aspect=15, height=.5, palette=pal)

g.map(sns.kdeplot, "Score",
      bw_adjust=.5, clip_on=False,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "Score", clip_on=False, color="w", lw=2, bw_adjust=.5)
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)
g.map(label, "Score")
g.figure.subplots_adjust(hspace=-.25)
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

Далее необходимо подготовить данные для алгоритмов.
Преобразуем зависимые данные из строки в числа:

In [ ]:
factor = pd.factorize(dataset['Action'])

In [ ]:
dataset.Action = factor[0]
definitions = factor[1]

In [ ]:
dataset.head()

Разделим массив данных на зависимые и независимые:

In [ ]:
X = dataset.iloc[:,0:1].values
y = dataset.iloc[:,1].values

In [ ]:
print(X[:5,:])
print(y[:15])

Далее разбиваем массив данных на части.
Одна часть будет использована для обучения алгоритмов, другая для проверки работы.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 10)


In [ ]:
print(X_train[:5])
print(X_test[:5])

Преобразуем независимые данные в более удобный для алгоритмов вид:

In [ ]:
minmax_scaler = MinMaxScaler()
X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)

In [ ]:
print(X_train[:5])
print(X_test[:5])

Далее создадим модели нескольких алгоритмов, пригодных для классификации:

In [ ]:
classifiers = []

SVC_model = svm.SVC()
classifiers.append(SVC_model)
KNN_model = KNeighborsClassifier()
classifiers.append(KNN_model)
DT_model = tree.DecisionTreeClassifier()
classifiers.append(DT_model)
RF_model = RandomForestClassifier()
classifiers.append(RF_model)
GNB_model = GaussianNB()
classifiers.append(GNB_model)

И в завершении, для каждой созданной модели:
1. Проведём ее обучение
2. Проверим работу на тестовых данных.
3. Выведем полученные метрики.

In [ ]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy of {0} is {1}".format(clf.__class__, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of {0} is\n {1}\n".format(clf.__class__, cm))
    print("Classification report:\n{0}".format(classification_report(y_test, y_pred, zero_division=0)))


Из полученных результатов сделаем вывод, что лучше всего с данной задачей справилась модель
`C-Support Vector Classification.`
